In [17]:
import numpy as np
import pandas as pd

In [18]:
fa = lambda x: x**3 - 2*x - 5
dfa = lambda x: 3*x**2 - 2

fb = lambda x: np.exp(-x) - x
dfb = lambda x: -np.exp(-x) - 1

fc = lambda x: x * np.sin(x) - 1
dfc = lambda x: np.sin(x) + x * np.cos(x)

In [ ]:
iter_a = lambda x: x - fa(x) / dfa(x)
iter_b = lambda x: x - fb(x) / dfb(x)
iter_c = lambda x: x - fc(x) / dfc(x)

def solve_iter(iter_fun, x, max_iter=30, cur_iter=0):
    if cur_iter == max_iter:
        return x
    return solve_iter(iter_fun, iter_fun(x), max_iter, cur_iter + 1)

solutions = pd.DataFrame([solve_iter(iter_a, 2), solve_iter(iter_b, 0.5), solve_iter(iter_c, 1)])

real_solutions = np.array([2.0945514815423, 0.56714352628852, 1.1141571379514], dtype=np.double)

def solve_iter_with_precision(iter_fun, x0, tolerance=1e-7, max_iter=100):
    x_prev = x0
    x_curr = iter_fun(x_prev)
    iteration = 1
    while abs(x_curr - x_prev) > tolerance and iteration < max_iter:
        x_prev = x_curr
        x_curr = iter_fun(x_prev)
        iteration += 1
    return x_curr, iteration

solutions4bit = [solve_iter_with_precision(iter_a, np.double(2), np.pow(1/2, 4)), 
    solve_iter_with_precision(iter_b, np.double(0.5), np.pow(1/2, 4)),
    solve_iter_with_precision(iter_c, np.double(1), np.pow(1/2, 4))]

In [37]:
solutions24bit = pd.DataFrame(
    [solve_iter_with_precision(iter_a, solutions4bit[0][0], np.pow(1/2, 24)), 
    solve_iter_with_precision(iter_b, solutions4bit[1][0], np.pow(1/2, 24)),
    solve_iter_with_precision(iter_c, solutions4bit[2][0], np.pow(1/2, 24))]
)

solutions24bit

,0,1
0,2.094551,2
1,0.567143,2
2,1.114157,1


In [38]:
solutions53bit = pd.DataFrame(
    [solve_iter_with_precision(iter_a, solutions4bit[0][0], np.pow(1/2, 53)), 
    solve_iter_with_precision(iter_b, solutions4bit[1][0], np.pow(1/2, 53)),
    solve_iter_with_precision(iter_c, solutions4bit[2][0], np.pow(1/2, 53))]
)

solutions53bit

,0,1
0,2.094551,3
1,0.567143,3
2,1.114157,100
